# **Homework 4** Problem 1

## (a) Prepare code and dataset

### Download source code

In [ ]:
!git clone https://github.com/princeton-vl/pytorch_stacked_hourglass.git

### Download MPII dataset

In [ ]:
!wget https://datasets.d2.mpi-inf.mpg.de/andriluka14cvpr/mpii_human_pose_v1.tar.gz

In [ ]:
!tar -xvzf mpii_human_pose_v1.tar.gz -C pytorch_stacked_hourglass/data/MPII/

### Visualize some images in MPII dataset

In [ ]:
# TODO

## (c) Train the network

Train with one stack (1HG). Please keep your terminal output in order to get full credit.

In [ ]:
%cd pytorch_stacked_hourglass/
!python train.py -e test_run_001 --max_iters 4

Draw your 1HG loss plot from log file.

In [ ]:
# TODO

## (d) Inference and Visualization

### Infer HPE using the pretrain model

Infer and evaluate the pretrained 2HG model. Please keep your terminal output in order to get full credit.

In [ ]:
!python test.py -c test_run_001

Infer and evaluate the pretrained 8HG model. Please keep your terminal output in order to get full credit.

In [ ]:
# TODO

Visualize some human pose estimation results.

In [ ]:
# TODO

### Try customized images

Download some images from the internet and infer and visualize the human pose.

In [ ]:
# TODO

## (e) Install VideoPose3D
### Pytorch & Caffe2

In [ ]:
!wget https://anaconda.org/pytorch/pytorch/1.2.0/download/linux-64/pytorch-1.2.0-py3.6_cuda10.0.130_cudnn7.6.2_0.tar.bz2
!tar xvjf pytorch-1.2.0-py3.6_cuda10.0.130_cudnn7.6.2_0.tar.bz2
!cp -r lib/python3.6/site-packages/* /usr/local/lib/python3.6/dist-packages/

# It might take some time

### Check if Caffe2 was built


In [ ]:
# To check if Caffe2 build was successful
!python -c 'from caffe2.python import core' 2>/dev/null && echo "Success" || echo "Failure"

# To check if Caffe2 GPU build was successful
!python -c 'from caffe2.python import workspace; print(workspace.NumCudaDevices())'

In [ ]:
# Install COCO Dataset 

!apt-get install python-dev
!pip install cython
!pip install pycocotools
!git clone https://github.com/cocodataset/cocoapi.git
!cd cocoapi/PythonAPI && make install

import os
os.environ['COCOAPI'] = ":/content/cocoapi"

In [ ]:
# Install Detectron

!git clone https://github.com/facebookresearch/detectron
!pip install -r detectron/requirements.txt
!cd detectron && make
!python detectron/detectron/tests/test_spatial_narrow_as_op.py

### Install VideoPose3D & Copy Video Script to Detectron Tools Folder

In [ ]:
# copy file from VideoPose3d
!git clone https://github.com/facebookresearch/VideoPose3D
!cp VideoPose3D/inference/infer_video.py detectron/tools/infer_video.py

### Download Pretrained Human3.6m Coco Model

In [ ]:
!mkdir VideoPose3D/checkpoint
os.chdir('VideoPose3D/checkpoint')
!wget https://dl.fbaipublicfiles.com/video-pose-3d/pretrained_h36m_detectron_coco.bin
os.chdir('../..')

### Download Youtube Video for 3D Pose Estimation (specify YOUTUBE_ID)

In [ ]:
YOUTUBE_ID ='UpH7rm0cYbM'


!pip install -q youtube-dl
#download video
!youtube-dl -f 'bestvideo[ext=mp4]' --output "youtube.%(ext)s" https://www.youtube.com/watch?v=$YOUTUBE_ID

!mkdir videos   
  
# cut the first 5 seconds
!ffmpeg -y -loglevel info -i youtube.mp4 -ss 00:00:42 -t 00:00:05 videos/video.mp4

### Example: Compute 2D Coordinates with Detectron

In [ ]:
!mkdir output
!python detectron/tools/infer_video.py \
    --cfg detectron/configs/12_2017_baselines/e2e_keypoint_rcnn_R-101-FPN_s1x.yaml \
    --output-dir output \
    --image-ext mp4 \
    --wts https://dl.fbaipublicfiles.com/detectron/37698009/12_2017_baselines/e2e_keypoint_rcnn_R-101-FPN_s1x.yaml.08_45_57.YkrJgP6O/output/train/keypoints_coco_2014_train:keypoints_coco_2014_valminusminival/generalized_rcnn/model_final.pkl \
   videos

### Example: Prepare Detectron Output to fit VideoPose3D Input

In [ ]:
!mkdir ./VideoPose3D/data/detectronoutput
!cp output/video.mp4.npz VideoPose3D/data/detectronoutput/video.mp4.npz
os.chdir('VideoPose3D/data') # This script must be launched from the "data" directory
!python prepare_data_2d_custom.py -i detectronoutput -o myvideos
os.chdir('../../')

### Example: Compute 3D Joints with VideoPose3D

In [ ]:
!cp ./videos/video.mp4 VideoPose3D/video.mp4
os.chdir('VideoPose3D')

!python run.py -d custom -k myvideos -arc 3,3,3,3,3 -c checkpoint --evaluate pretrained_h36m_detectron_coco.bin --render --viz-subject video.mp4 --viz-action custom --viz-camera 0 --viz-video video.mp4 --viz-output output.mp4 --viz-export outputfile --viz-size 6

## (f) Questions

(1) Use the provided video and the pre-trained model of BOTH Stacked Hourglass and Detectron to do the inference and visualize the 2D human pose estimation results.


In [ ]:
# TODO


(2) Infer the 3D keypoints based on the 2D keypoints results from the two methods in (1). (needs to be displayed in Colab with mp4 format, sample codes and images are given below.)


In [ ]:
# display video
def show_local_mp4_video(file_name, width=640, height=480):
  import io
  import base64
  from IPython.display import HTML
  video_encoded = base64.b64encode(io.open(file_name, 'rb').read())
  return HTML(data='''<video width="{0}" height="{1}" alt="test" controls>
                        <source src="data:video/mp4;base64,{2}" type="video/mp4" />
                      </video>'''.format(width, height, video_encoded.decode('ascii')))

show_local_mp4_video('output.mp4', width=960, height=720)

(3) Besides the quality of the 2D keypoint detector, what other parameters or model architecture do you think that may affect the 3D performance? Explain why. (e.g., receptive fields, numbers of residual blocks, etc. )
